In [ ]:
# 파이썬 2와 파이썬 3 지원
from __future__ import division, print_function, unicode_literals

# 공통
import numpy as np
import os

# 일관된 출력을 위해 유사난수 초기화
np.random.seed(42)

# 맷플롯립 설정
%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt
plt.rcParams['axes.labelsize'] = 14
plt.rcParams['xtick.labelsize'] = 12
plt.rcParams['ytick.labelsize'] = 12

# 한글출력
plt.rcParams['font.family'] = 'D2Coding'
plt.rcParams['axes.unicode_minus'] = False

8. （3장에서 소개한） MNIST 데이터를 불러들여 훈련 세트, 검증 세트, 테스트 세트로 나눕
니다（예를 들면 훈련에 50,000개 샘플, 검증에 10,000개 샘플, 테스트에 10,000개 샘
플） . 그런 다음 랜덤 포레스트 분류기 , 엑스트라 트리 분류기, SVM 분류기 같은 여러 종
류의 분류기를 훈련시킵니다. 그리고 검증 세트에서 개개의 분류기보다 더 높은 성능을 내
도록 이들을 간접 또는 직접 투표 방법을 사용해 앙상블로 연결해보세요. 앙상블을 얻고
나면 테스트 세트로 확인해보세요. 개개의 분류기와 비교해서 성능이 얼마나 향상되나요?

In [2]:
from sklearn.datasets import fetch_openml
mnist = fetch_openml('mnist_784',version=1)

In [3]:
from sklearn.model_selection import train_test_split

In [4]:
X_train_val, X_test, y_train_val, y_test = train_test_split(mnist.data, mnist.target, test_size=10000,random_state=42)
X_train,X_val,y_train,y_val = train_test_split(X_train_val,y_train_val,test_size=10000,random_state=42)


In [5]:
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier
from sklearn.svm import LinearSVC
from sklearn.neural_network import MLPClassifier

In [7]:
from random import random


random_forest_clf=RandomForestClassifier(n_estimators=10,random_state=42)
extra_tree_clf=ExtraTreesClassifier(n_estimators=10,random_state=42)
svm_clf=LinearSVC(max_iter=10000,random_state=42)
mlp_clf=MLPClassifier(random_state=42)

In [9]:
estimators = [random_forest_clf,extra_tree_clf,svm_clf,mlp_clf]
for estimator in estimators:
    print("훈련 예측기: ",estimator)
    estimator.fit(X_train,y_train)

훈련 예측기:  RandomForestClassifier(n_estimators=10, random_state=42)
훈련 예측기:  ExtraTreesClassifier(n_estimators=10, random_state=42)
훈련 예측기:  LinearSVC(max_iter=10000, random_state=42)


c:\Users\nmh90\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\svm\_base.py:1225: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


훈련 예측기:  MLPClassifier(random_state=42)


In [10]:
[estimator.score(X_val,y_val) for estimator in estimators]

[0.9469, 0.9492, 0.8558, 0.9626]

In [11]:
from sklearn.ensemble import VotingClassifier

In [12]:
named_estimators = [
    ("random_forest_clf",random_forest_clf),
    ("extra_trees_clf",extra_tree_clf),
    ("svm_clf",svm_clf),
    ("mlp_clf",mlp_clf),
]

In [13]:
voting_clf = VotingClassifier(named_estimators)

In [14]:
voting_clf.fit(X_train,y_train)

In [ ]:
voting_clf.score(X_val,y_val)

In [ ]:
[estimator.score(X_val,y_val) for estimator in voting_clf.estimators_]

In [ ]:
voting_clf.set_params(svm_clf=None)

In [ ]:
voting_clf.estimators

In [ ]:
voting_clf.estimators_

In [ ]:
del voting_clf.estimators_[2]

In [ ]:
voting_clf.score(X_val,y_val)

In [ ]:
voting_clf.voting="soft"

In [ ]:
voting_clf.score(X_val,y_val)

In [ ]:
voting_clf.score(X_test,y_test)

In [ ]:
[esitmator.score(X_test,y_test) for estimator in voting_clf.estimators_]

9. 이전 연습문제의 각 분류기를 실행해서 검증 세트에서 예측을 만들고 그 결과로 새로운 훈
련 세트를 만들어보세요. 각 훈련 샘플은 하나의 이미지에 대한 전체 분류기의 예측을 담
은 벡터이고 타깃은 이미지의 클래스입니다. 새로운 훈련 세트에 분류기 하나를 훈련시켜
보세요. 축하합니다. 방금 블렌더를 훈련시켰습니다. 그리고 이 분류기를 모아서 스태킹
앙상블을 구성했습니다! 이제 테스트 세트에 앙상블을 평가해보세요. 테스트 세트의 각 이
미지에 대해 모든 분류기로 예측을 만들고 앙상블의 예측 결과를 만들기 위해 블렌더에 그
예측을 주입합니다. 앞서 만든 투표 분류기와 비교하면 어떤가요?


In [ ]:
X_val_predictions = np.empty((len(X_val), len(estimators)), dtype=np.float32)

for index, estimator in enumerate(estimators):
    X_val_predictions[:, index] = estimator.predict(X_val)

In [ ]:
X_val_predictions


In [ ]:
rnd_forest_blender = RandomForestClassifier(n_estimators=200, oob_score=True, random_state=42)
rnd_forest_blender.fit(X_val_predictions, y_val)

In [ ]:
rnd_forest_blender.oob_score_


In [ ]:
X_test_predictions = np.empty((len(X_test), len(estimators)), dtype=np.float32)

for index, estimator in enumerate(estimators):
    X_test_predictions[:, index] = estimator.predict(X_test)

In [ ]:
from sklearn.metrics import accuracy_score


In [ ]:
accuracy_score(y_test, y_pred)
